In [9]:
from skimpy import skim as sk
import json 
import requests 
import pandas as pd
from pandas import json_normalize 
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from dev_tools import autoclean_dataset


In [8]:
df_degradations =  pd.read_csv('D:\\Sky_Analytics\\Datasets\\df_degradations\\degradations.csv')
df_degradations.head()

,ref_deg,linked_aero,compo_concerned,usure_nouvelle,measure_day,need_replacement
0,D003661,E170_6353,REAE170-E170_6353-0,53.308375,2024-06-01,False
1,D009402,E170_6353,REAE170-E170_6353-1,50.111950,2024-06-01,False
2,D000812,E170_6353,SYSE170-E170_6353-2,16.730575,2024-06-01,False
3,D006665,E170_6353,ORDE170-E170_6353-3,6.653075,2024-06-01,False
4,D001163,E170_6353,AUTE170-E170_6353-4,42.033525,2024-06-01,False


In [7]:
autoclean_dataset(df_degradations)

TypeError: argument of type 'method' is not iterable